In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
rating_df = pd.read_csv("data/ml-25m/ratings.csv")
rating_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [3]:
movies_df = pd.read_csv("data/ml-25m/movies.csv")

In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
no_items = movies_df['movieId'].max()

In [6]:
rating_df.shape

(25000095, 4)

In [7]:
no_users = rating_df['userId'].max()

In [8]:
# sort dataset by id and timestamp
rating_df = rating_df.sort_values(['userId', 'timestamp'])

In [9]:
# split dataset in train and eval set

In [9]:
def func1(df):
    n_rows = df.shape[0]
    df['percentile'] = np.linspace(0, 100, n_rows)
    return df

rating_df = rating_df.groupby('userId').apply(func1).reset_index(drop=True)

In [10]:
train_size = 80 # percent
train_data = rating_df[rating_df['percentile'] <= train_size]
test_data = rating_df[rating_df['percentile'] > train_size]

In [11]:
train_data.shape, test_data.shape

((19968907, 5), (5031188, 5))

In [12]:
del train_data['percentile'], test_data['percentile']

In [13]:
train_data.head()

,userId,movieId,rating,timestamp
0,1,5952,4.0,1147868053
1,1,2012,2.5,1147868068
2,1,2011,2.5,1147868079
3,1,1653,4.0,1147868097
4,1,1250,4.0,1147868414


In [14]:
train_data['rating'].value_counts()

rating
4.0    5346555
3.0    3892905
5.0    3006138
3.5    2476949
4.5    1743999
2.0    1297363
2.5     976141
1.0     611705
1.5     310786
0.5     306366
Name: count, dtype: int64

In [15]:
# assuming rating more than 3 means user like the movie else he/she didn't
rating_threshold = 3
from data import Data
from tqdm import tqdm

In [16]:
trainds = Data(train_data)
testds = Data(test_data)

In [101]:
batch_size = 2**14
n_workers = 4

traindl = torch.utils.data.DataLoader(trainds, batch_size=batch_size, shuffle=True, num_workers=1)

testdl = torch.utils.data.DataLoader(testds, batch_size=batch_size, shuffle=True, num_workers=1)
print(batch_size)

16384


In [160]:
item_feature_name = "movieId"
user_feature_name = "userId"

class ItemTower(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.features = config['item']['features']
        if item_feature_name in self.features:
            self.item_module = nn.Embedding(config['item']['n_items'], config['item']['emb'])
        else:
            self.item_module = nn.Identity()
        self.mlp = nn.Sequential(
            nn.Linear(config['item']['emb'], config['item']['emb']),
            nn.GELU(),
            nn.LayerNorm(config['item']['emb'])
        )
        
    def forward(self, batch):
        item_emb = self.item_module(batch[item_feature_name].long())
        return self.mlp(item_emb)


class UserTower(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.features = config['user']['features']
        if user_feature_name in self.features:
            self.user_module = nn.Embedding(config['user']['n_users'], config['user']['emb'])
        else:
            self.item_module = nn.Identity()
        self.mlp = nn.Sequential(
            nn.Linear(config['user']['emb'], config['user']['emb']),
            nn.GELU(),
            nn.LayerNorm(config['user']['emb'])
        )
        
    def forward(self, batch):
        user_emb = self.user_module(batch[user_feature_name].long())
        return self.mlp(user_emb)

class CombinerTower(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.item_tower = ItemTower(config)
        self.user_tower = UserTower(config)
        user_dim, item_dim = config['user']['emb'], config['item']['emb']
        self.mlp = nn.Sequential(
            nn.Linear(user_dim+item_dim, config['emb']),
            nn.GELU(),
            nn.LayerNorm(config['emb'])
        )
        self.tasks = nn.ModuleDict({
            task_name: nn.Linear(config['emb'], out) for task_name, out in config['tasks']
        })
        
    def forward(self, batch):
        item_emb = self.item_tower(batch)
        user_emb = self.user_tower(batch)
        x = torch.concat([user_emb, item_emb], axis=1)
        x = self.mlp(x)
        res = {}
        for task_name, mod in self.tasks.items():
            res[task_name] = mod(x)
        return res

config = {
    'user':{
        'features':[
            'userId'
        ],
        'n_users':no_users + 2**10,
        'emb': 64
    },
    'item':{
        'features':[
            'movieId'
        ],
        'n_items':no_items + 2**10,
        'emb': 64
    },
    'emb':64,
    'tasks':[
        ['rating', 1]
    ]
}

In [148]:
model = CombinerTower(config)

In [134]:
from torchmetrics.classification import average_precision, ROC, Recall, Accuracy

class Metric:
    def __init__(self, device=torch.device("cpu"), task="binary"):
        device = device
        self.counter = 0
        self._metric = {
            "map": average_precision.AveragePrecision(task=task),
            "accuracy": Accuracy(task=task),
            "recall": Recall(task=task)
        }
        self._accelerator = {
            "map": 0,
            "accuracy": 0,
            "recall": 0
        }
        
    def __call__(self, output, actual) -> dict:
        out = {}
        for metric_name, mod in self._metric.items():
            out[metric_name] = mod(output, actual)
            self._accelerator[metric_name] += out[metric_name]
        self.counter += 1
        return out
    
    def get(self):
        return {
            k:v/self.counter for k,v in self._accelerator.items()
        }

    def to(self, device):
        for i in self._metric:
            self._metric[i].to(device)
        return self

In [135]:
metric = Metric()
output, actual = torch.randn(20, 1).view(-1), torch.randint(0, 2, size=(20, 1)).view(-1)
print(actual, output)
print(metric(output, actual))
metric.get()

tensor([0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1]) tensor([-0.4142,  0.9052, -1.1952,  1.1407,  0.1014, -1.4244, -0.9388, -0.7567,
         0.1958, -0.9591,  0.6681,  0.6016, -0.3790, -2.0306,  0.4814, -0.2623,
         1.6520,  0.8246, -0.3324,  0.3609])
{'map': tensor(0.5258), 'accuracy': tensor(0.6000), 'recall': tensor(0.6250)}


{'map': tensor(0.5258), 'accuracy': tensor(0.6000), 'recall': tensor(0.6250)}

In [136]:
class Trainer:
    def __init__(self,
                model: torch.nn.Module,
                train_dataloader: torch.utils.data.DataLoader,
                eval_dataloader: torch.utils.data.DataLoader = None,
                loss_fn = torch.nn.functional.mse_loss,
                epochs: int = 5,
                optimizer_clz = torch.optim.SGD,
                optim_params: dict = {'lr':1e-2},
                device: torch.device = torch.device("cpu"),
                metric_collection = None,
                verbose = 1,
                log_step = 500,
                *args, **kwargs
                ):
        self.log_step = log_step
        self.verbose = verbose
        self.model = model.to(device)
        self.train_dl = train_dataloader
        self.eval_dl = eval_dataloader
        self.loss_fn = loss_fn
        self.optimizer = optimizer_clz(self.model.parameters(), **optim_params)
        self.epochs = epochs
        self.device = device
        self.metric_collection = metric_collection.to(device) if metric_collection else None

    def compute_loss(self, batch, output):
        loss = {}
        for k in output:
            if k in batch:
                y = (batch[k] >= rating_threshold).float()
                loss[k] = self.loss_fn(output[k], y.view(-1, 1))
        return loss

    def compute_metric(self, batch, output, show_tensor=False):
        res = {}
        for k in output:
            if k in batch:
                y = (batch[k] >= rating_threshold).long()
                if show_tensor:
                    print(batch[k].sum(), batch[k].shape[0])
                res[k] = self.metric_collection(output[k].view(-1), y.view(-1))
        return res
    
    def set_output(self, _iter, loss, avg_loss, train=True):
        _iter.set_description(
            "{}: step loss: {:.3f}, avg_loss: {:.3f}".format(
                "Train" if train else "Eval",
                loss,
                avg_loss
            )
        )
    def train_epoch(self):
        self.model = self.model.train()
        
        _iter = tqdm(self.train_dl)
        n_batches = len(self.train_dl)
        total_loss = 0
        for _i, batch in enumerate(_iter):
            self.optimizer.zero_grad()
            output = self.model(batch)
            loss = self.compute_loss(batch, output)
            # multiple task we need better way to gives weights, ryt now it is uniform
            loss = sum([loss[l] for l in loss])/(len(loss) + 1e-6)
            
            loss.backward()
            self.optimizer.step()
            total_loss += loss.cpu().item()
            
            self.set_output(_iter, loss.item(), total_loss/(_i+1), train=True)
            show_tensor = False
            if self.verbose > 0 \
                and _i % self.log_step == 0 and _i != 0:
                print(self.metric_collection.get())
                show_tensor = True
            with torch.no_grad():
                self.compute_metric(batch, output, show_tensor)
        return {
            'loss': total_loss/(n_batches),
            'mode': 'Train'
        }
    
    @torch.no_grad()
    def eval_epoch(self):
        self.model = self.model.eval()
        
        _iter = tqdm(self.eval_dl)
        n_batches = len(self.eval_dl)
        total_loss = 0
        for _i, batch in enumerate(_iter):
            output = self.model(batch)
            loss = self.compute_loss(batch, output)
            # multiple task we need better way to gives weights, ryt now it is uniform
            loss = sum([loss[l] for l in loss])/(len(loss) + 1e-6)
            
            total_loss += loss.cpu().item()
            self.set_output(_iter, loss.item(), total_loss/(_i+1), train=False)
            
            show_tensor = False
            if self.verbose > 0 \
                and _i % self.log_step == 0 and _i != 0:
                print(self.metric_collection.get())
                show_tensor = True
            self.compute_metric(batch, output, show_tensor=show_tensor)
                
        return {
            'loss': total_loss/(n_batches),
            'mode': 'Eval'
        }

    def fit(self):
        for epoch in range(1, self.epochs+1):
            print("EPOCH:", epoch)
            self.train_epoch()
            eval_output = self.eval_epoch()
            print(eval_output)

In [137]:
trainer = Trainer(
    model=model, train_dataloader=traindl, eval_dataloader=testdl, 
    loss_fn=F.binary_cross_entropy_with_logits,
    metric_collection = Metric(),
    log_step = 100
)

In [138]:
trainer.fit()

EPOCH: 1


Train: step loss: 0.491, avg_loss: 0.562:   8%|▊         | 101/1219 [01:58<19:36,  1.05s/it]

{'map': tensor(0.8256), 'accuracy': tensor(0.7400), 'recall': tensor(0.8694)}
tensor(58244.5000, dtype=torch.float64) 16384


Train: step loss: 0.479, avg_loss: 0.524:  16%|█▋        | 201/1219 [03:44<17:53,  1.05s/it]

{'map': tensor(0.8257), 'accuracy': tensor(0.7804), 'recall': tensor(0.9319)}
tensor(58206.5000, dtype=torch.float64) 16384


...

{'map': tensor(0.8404), 'accuracy': tensor(0.8201), 'recall': tensor(0.9982)}
tensor(56953.5000, dtype=torch.float64) 16384


Eval: step loss: 0.492, avg_loss: 0.488:  98%|█████████▊| 301/308 [06:35<00:09,  1.31s/it]

{'map': tensor(0.8403), 'accuracy': tensor(0.8199), 'recall': tensor(0.9982)}
tensor(56835., dtype=torch.float64) 16384


Eval: step loss: 0.495, avg_loss: 0.488: 100%|██████████| 308/308 [06:47<00:00,  1.32s/it]

{'loss': 0.4884836112524008, 'mode': 'Eval'}


#### Evaluation Metric

`{'map': tensor(0.8403), 'accuracy': tensor(0.8199), 'recall': tensor(0.9982)}`

In [139]:
torch.save(model.state_dict(), "./checkpoint/baseline_chk_v1.pt")

In [143]:
model.item_tower({"movieId":torch.tensor([100]).view(1,1)})

torch.Size([1, 1, 64])

In [144]:
#### Generate embeddings

1

In [159]:
import torch
import pandas as pd

In [161]:
model = CombinerTower(config)

model.load_state_dict(torch.load("./checkpoint/baseline_chk_v1.pt"))

<All keys matched successfully>

In [162]:
movie_df = pd.read_csv("./data/ml-25m/movies.csv")

In [163]:
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [164]:
movie_df.shape

(62423, 3)

In [169]:
emb_dim = 64
with torch.no_grad():
    movie_df['embedding'] = movie_df['movieId'].map(lambda x: model.item_tower({"movieId": torch.tensor([x])}).view(emb_dim))

In [177]:
emb = torch.stack(movie_df['embedding'].values.tolist(), axis=0)

In [184]:
emb = emb/(emb.norm(dim=1).view(-1,1) + 1e-7)

In [190]:
movie_df['norm_embedding'] = emb.unbind(dim=0)

In [191]:
movie_df.head()

,movieId,title,genres,embedding,norm_embedding
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[tensor(-0.6897), tensor(1.1606), tensor(0.659...","[tensor(-0.0859), tensor(0.1445), tensor(0.082..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[tensor(-0.8724), tensor(0.3376), tensor(1.183...","[tensor(-0.1086), tensor(0.0420), tensor(0.147..."
2,3,Grumpier Old Men (1995),Comedy|Romance,"[tensor(-0.6817), tensor(-0.0421), tensor(-0.7...","[tensor(-0.0848), tensor(-0.0052), tensor(-0.0..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[tensor(-0.6069), tensor(-0.3672), tensor(-0.6...","[tensor(-0.0757), tensor(-0.0458), tensor(-0.0..."
4,5,Father of the Bride Part II (1995),Comedy,"[tensor(-0.7473), tensor(-0.8992), tensor(2.59...","[tensor(-0.0932), tensor(-0.1122), tensor(0.32..."


In [222]:
import random

25478

In [264]:
def get_similar(movie_df, movie_id=None, k=20):
    if movie_id is None:
        movie_id = np.random.randint(1, movie_df.shape[0])        
    query = movie_df[movie_df['movieId'] == movie_id].iloc[0]
    query_emb = query['norm_embedding']
    sim_scores = (emb @ query_emb.view(1,-1).T)
    sorted_idx = torch.argsort(sim_scores.view(-1),descending=True)
    top_k_idx = sorted_idx[1:k+1]
    res = movie_df.iloc[top_k_idx]
    titles = res['title'].values
    genres = res['genres'].values
    scores = sim_scores[top_k_idx]
    return {
        "query": query[['movieId', 'title', 'genres']].to_dict(),
        'result': [[t,g,s] for t,g,s in zip(titles, genres, scores)]
    }

In [265]:
get_similar(movie_df, movie_id=192389, k=10)

{'query': {'movieId': 192389,
  'title': 'Venom (2018)',
  'genres': 'Action|Horror|Sci-Fi|Thriller'},
 'result': [['Cynthia (1947)', 'Comedy|Drama', tensor([0.6378])],
  ['The Sandpit Generals (1971)', '(no genres listed)', tensor([0.6098])],
  ['Mask of Dust (1954)', 'Drama', tensor([0.6071])],
  ['Twist Around The Clock (1961)', '(no genres listed)', tensor([0.6045])],
  ['Claudine (1974)', 'Drama', tensor([0.6001])],
  ['Second Fiddle to a Steel Guitar (1966)', 'Comedy', tensor([0.5996])],
  ["The Last Sharknado: It's About Time (2018)",
   'Action|Adventure|Comedy|Fantasy|Sci-Fi|Thriller',
   tensor([0.5921])],
  ['Reivers, The (1969)', 'Comedy|Drama', tensor([0.5906])],
  ['Rack, The (1956)', 'Drama|War', tensor([0.5825])],
  ['Wonderful Days (a.k.a. Sky Blue) (2003)',
   'Animation|Sci-Fi',
   tensor([0.5800])]]}

In [263]:
movie_df.iloc[55643]

movieId                                                      192389
title                                                  Venom (2018)
genres                                Action|Horror|Sci-Fi|Thriller
embedding         [tensor(0.5151), tensor(-0.9153), tensor(-0.55...
norm_embedding    [tensor(0.0641), tensor(-0.1138), tensor(-0.06...
Name: 55643, dtype: object

In [260]:
movie_df[movie_df['title'].map(lambda x: "venom" in x.lower())]

,movieId,title,genres,embedding,norm_embedding
6033,6145,Venom (1982),Horror|Thriller,"[tensor(1.1496), tensor(-0.7547), tensor(-0.71...","[tensor(0.1434), tensor(-0.0941), tensor(-0.08..."
8790,26399,Five Deadly Venoms (1978),Action,"[tensor(-0.0652), tensor(-0.6690), tensor(-0.4...","[tensor(-0.0081), tensor(-0.0835), tensor(-0.0..."
10205,36531,Venom (2005),Horror|Thriller,"[tensor(-0.0295), tensor(-0.4241), tensor(1.43...","[tensor(-0.0037), tensor(-0.0528), tensor(0.17..."
22599,115727,Crippled Avengers (Can que) (Return of the 5 D...,Action|Adventure,"[tensor(0.3127), tensor(-0.1394), tensor(-0.73...","[tensor(0.0389), tensor(-0.0173), tensor(-0.09..."
32508,141070,Venom (1966),Drama,"[tensor(2.8667), tensor(0.3400), tensor(1.6905...","[tensor(0.3571), tensor(0.0424), tensor(0.2106..."
37355,152467,Venomous (2002),Horror|Sci-Fi,"[tensor(-0.9478), tensor(-1.0008), tensor(1.29...","[tensor(-0.1186), tensor(-0.1252), tensor(0.16..."
37357,152471,Silent Venom (2009),Action|Adventure|Horror|Thriller,"[tensor(-0.9665), tensor(1.5667), tensor(0.971...","[tensor(-0.1205), tensor(0.1954), tensor(0.121..."
55072,191077,Venom and Eternity (1951),(no genres listed),"[tensor(0.8810), tensor(0.1856), tensor(0.1019...","[tensor(0.1101), tensor(0.0232), tensor(0.0127..."
55643,192389,Venom (2018),Action|Horror|Sci-Fi|Thriller,"[tensor(0.5151), tensor(-0.9153), tensor(-0.55...","[tensor(0.0641), tensor(-0.1138), tensor(-0.06..."
56211,193669,Venom Islands (2012),Documentary,"[tensor(-0.0993), tensor(-0.6386), tensor(-0.4...","[tensor(-0.0124), tensor(-0.0797), tensor(-0.0..."
